In [1]:
from qiskit.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import HartreeFock, UCCSD
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit_nature.drivers import UnitsType
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

# Running VQE on a Statevector Simulator 

We demonstrate the calculation of the ground state energy for LiH at various interatomic distances. 

A driver for the molecule must be created at each such distance. 

Note that in this experiment, to reduce the number of qubits used, we freeze the core and remove two unoccupied orbitals. 

First, we define a function that takes an interatomic distance and returns the appropriate qubit operator,  
H, as well as some other information about the operator.

In [21]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    #repulsion_energy = molecule.nuclear_repulsion_energy
    #num_particles = molecule.num_alpha + molecule.num_beta
    #num_spin_orbitals = molecule.num_orbitals * 2
    #remove_list = [x % molecule.num_orbitals for x in remove_list]
    #freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    #remove_list = [x - len(freeze_list) for x in remove_list]
    #remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    #freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOp(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    #ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    #num_spin_orbitals -= len(freeze_list)
    #num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    #num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    #qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    #shift = energy_shift + repulsion_energy
    #return qubitOp, num_particles, num_spin_orbitals, shift
    return qubitOp

### First, the exact ground state energy is calculated using the qubit operator and a classical exact eigensolver. Subsequently, the initial state  
|ψ⟩
  is created, which the VQE instance uses to produce the final ansatz  
min
θ
(
|
ψ
(
θ
)
⟩
)
 . The exact result and the VQE result at each interatomic distance is stored. Observe that the result given by vqe.run(backend)['energy'] + shift is equivalent the quantity  
min
θ
(
⟨
ψ
(
θ
)
|
H
|
ψ
(
θ
)
⟩
)
 , where the minimum is not necessarily the global minimum.

When initializing the VQE instance with VQE(qubitOp, var_form, optimizer, 'matrix') the expectation value of  
H
  on  
|
ψ
(
θ
)
⟩
  is directly calculated through matrix multiplication. However, when using an actual quantum device, or a true simulator such as the qasm_simulator with VQE(qubitOp, var_form, optimizer, 'paulis') the calculation of the expectation value is more complicated. A Hamiltonian may be represented as a sum of a Pauli strings, with each Pauli term acting on a qubit as specified by the mapping being used. Each Pauli string has a corresponding circuit appended to the circuit corresponding to  
|
ψ
(
θ
)
⟩
 . Subsequently, each of these circuits is executed, and all of the results are used to determine the expectation value of  
H
  on  
|
ψ
(
θ
)
⟩
 . In the following example, we initialize the VQE instance with matrix mode, and so the expectation value is directly calculated through matrix multiplication.

Note that the following code snippet may take a few minutes to run to completion.

In [22]:
dist = 0.5
qubitOp = get_qubit_op(dist)
print(type(qubitOp))

AttributeError: 'ElectronicStructureDriverResult' object has no attribute 'one_body_integrals'

In [7]:
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")

AttributeError: 'ElectronicStructureDriverResult' object has no attribute 'nuclear_repulsion_energy'